In [1]:
import json
import pymongo
import pandas as pd
import numpy as np

In [2]:
#https://www.geeksforgeeks.org/how-to-fetch-data-from-mongodb-using-python/

client = pymongo.MongoClient("mongodb://localhost:27017/")
 
# Database Name
db = client["cric_stats"]
 
# Collection Name
col = db["data"]
 
x_innings = col.find_one()["innings"]

#print(json.dumps(x_innings, indent=4, sort_keys=True))

In [3]:
def per_over_json_to_df(x, team):
    
    delivery_dict = x['deliveries']
    
    x_df_rows = pd.DataFrame()
    
    x_df = dict()
    
    x_dict_over = x['over']
    
    count_balls = 0
    
    for x_dict in delivery_dict:
        
        if 'batter' in x_dict.keys():
            
            x_df['batter'] = x_dict['batter']
            
        else:
            
            x_df['batter'] = None
            
        if 'bowler' in x_dict.keys():
            
            x_df['bowler'] = x_dict['bowler']
            
        else:
            
            x_df['bowler'] = None
            
        if 'non_striker' in x_dict.keys():
            
            x_df['non_striker'] = x_dict['non_striker']
            
        else:
            
            x_df['non_striker'] = None
            
        if 'runs' in x_dict.keys():
            
            x_df['runs'] = x_dict['runs']['batter']
            
            x_df['extras'] = x_dict['runs']['extras']
            
            x_df['total'] = x_dict['runs']['total']
            
            if 'wickets' in x_dict.keys():

                x_df['wickets_kind'] = x_dict['wickets'][0]['kind']

                x_df['wickets_player_out'] = x_dict['wickets'][0]['player_out']

            else:

                x_df['wickets_kind'] = None

                x_df['wickets_player_out'] = None
            
        else:
            
            x_df['runs'] = 0
            
            x_df['extras'] = 0
            
            x_df['total'] = 0
        
        x_df = pd.DataFrame(x_df,index=[count_balls])
        
        count_balls += 1
        
        #print(x_df_rows.shape)
            
        if x_df_rows.shape[0]>0:
            
            x_df_rows = pd.concat([x_df_rows, x_df],ignore_index=True)
        
        else:
            
            x_df_rows = x_df
        
        x_df = dict()
            
    x_df_rows['over'] = x_dict_over
    
    x_df_rows['balls'] = range(x_df_rows.shape[0])
    
    x_df_rows['team'] = team
    
    return x_df_rows

In [4]:
x_innings_df = pd.DataFrame()

for x_dict in x_innings:
    
    team = x_dict['team']
    
    print(team)
    
    for x_dict_over in x_dict['overs']:
    
        x_df_rows = per_over_json_to_df(x_dict_over, team)
    
        if x_innings_df.shape[0]>0:

            x_innings_df = pd.concat([x_innings_df,x_df_rows],ignore_index=True)

        else:

            x_innings_df = x_df_rows
            
        #print(x_innings_df.shape)

Middlesex
Sussex
Middlesex


In [5]:
n_over = 20

x_innings_df.iloc[6*(n_over-1):6*n_over,:]

,batter,bowler,non_striker,runs,extras,total,wickets_kind,wickets_player_out,over,balls,team
114,DJ Malan,JE Anyon,NJ Dexter,0,0,0,None,None,19,0,Middlesex
115,DJ Malan,JE Anyon,NJ Dexter,0,0,0,None,None,19,1,Middlesex
116,DJ Malan,JE Anyon,NJ Dexter,0,2,2,None,None,19,2,Middlesex
117,DJ Malan,JE Anyon,NJ Dexter,0,0,0,None,None,19,3,Middlesex
118,DJ Malan,JE Anyon,NJ Dexter,0,0,0,None,None,19,4,Middlesex
119,DJ Malan,JE Anyon,NJ Dexter,0,0,0,None,None,19,5,Middlesex


In [6]:
x_innings[0]['overs'][n_over-1]

{'over': 19,
 'deliveries': [{'batter': 'DJ Malan',
   'bowler': 'JE Anyon',
   'non_striker': 'NJ Dexter',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'DJ Malan',
   'bowler': 'JE Anyon',
   'non_striker': 'NJ Dexter',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'DJ Malan',
   'bowler': 'JE Anyon',
   'extras': {'legbyes': 2},
   'non_striker': 'NJ Dexter',
   'runs': {'batter': 0, 'extras': 2, 'total': 2}},
  {'batter': 'DJ Malan',
   'bowler': 'JE Anyon',
   'non_striker': 'NJ Dexter',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'DJ Malan',
   'bowler': 'JE Anyon',
   'non_striker': 'NJ Dexter',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}},
  {'batter': 'DJ Malan',
   'bowler': 'JE Anyon',
   'non_striker': 'NJ Dexter',
   'runs': {'batter': 0, 'extras': 0, 'total': 0}}]}

In [7]:
col.find_one()['info']

{'balls_per_over': 6,
 'city': 'Brighton',
 'dates': ['2014-04-06', '2014-04-07', '2014-04-08', '2014-04-09'],
 'event': {'name': 'LV= County Championship', 'group': 1},
 'gender': 'male',
 'match_type': 'MDM',
 'officials': {'umpires': ['NL Bainton', 'NGC Cowley']},
 'outcome': {'winner': 'Sussex', 'by': {'innings': 1, 'runs': 127}},
 'players': {'Middlesex': ['CJL Rogers',
   'SD Robson',
   'DJ Malan',
   'NJ Dexter',
   'EJG Morgan',
   'AM Rossington',
   'JA Simpson',
   'JAR Harris',
   'TS Roland-Jones',
   'TJ Murtagh',
   'ST Finn'],
  'Sussex': ['EC Joyce',
   'LWP Wells',
   'MH Yardy',
   'MJ Prior',
   'MW Machan',
   'Ashar Zaidi',
   'BC Brown',
   'SA Piolet',
   'JE Anyon',
   'SJ Magoffin',
   'J Lewis']},
 'registry': {'people': {'AM Rossington': '7bcde3fe',
   'Ashar Zaidi': '9bab2b81',
   'BC Brown': '8af86756',
   'CJL Rogers': 'd8360178',
   'DJ Malan': 'ad9c32a2',
   'EC Joyce': 'a48fc79d',
   'EJG Morgan': 'd2a6c0e6',
   'J Lewis': '6c75310b',
   'JA Simpson':

In [8]:
#summarize the batting scorecard

batting_scorecard_df = x_innings_df.groupby(['team','batter'])['runs'].sum().reset_index()

In [9]:
batting_scorecard_df

#check reference match scorecard
#https://www.espncricinfo.com/series/lv-county-championship-division-one-2014-691689/sussex-vs-middlesex-692723/full-scorecard

,team,batter,runs
0,Middlesex,AM Rossington,12
1,Middlesex,CJL Rogers,45
2,Middlesex,DJ Malan,71
3,Middlesex,EJG Morgan,2
4,Middlesex,JA Simpson,7
5,Middlesex,JAR Harris,16
6,Middlesex,NJ Dexter,11
7,Middlesex,SD Robson,12
8,Middlesex,ST Finn,32
9,Middlesex,TJ Murtagh,9


In [ ]:
import sqlite3 as sql

In [ ]:
# get output ready for database export
conn = sql.connect('sussex_middlesex_6_Apr_2014_Test_Champ.db')
x_innings_df.to_sql('sussex_middlesex_6_Apr_2014_Test_Champ', conn)